In [1]:
%%bash
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"
wget ${DATA_URL}
wget ${MODEL_URL}

--2025-12-16 22:40:16--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-17T07%3A34%3A40Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-17T06%3A34%3A07Z&ske=2025-12-17T07%3A34%3A40Z&sks=b&skv=2018-11-09&sig=oUObHGlaoEF%2BIlffS0%2F6YKBsxb%2ByNJyxzuYEHslQdMk%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTk1NTQxNywibmJmIjoxNzY1OTUzNjE3L

In [2]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00


In [3]:
import onnxruntime as ort

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [4]:
input_name

'input'

In [5]:
output_name

'output'

In [6]:
!pip install pillow

In [7]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [8]:
!pip install keras-image-helper

In [9]:
from torchvision import transforms

transform_pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [10]:
image = download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")

In [11]:
resized_image = prepare_image(image, (200,200))

In [12]:
image_final = transform_pipeline(resized_image)

In [13]:
image_final

tensor([[[-1.0733, -1.0048, -1.0390,  ..., -1.0733, -1.0733, -1.2103],
         [-1.0562, -1.0219, -1.0048,  ..., -1.0219, -1.0562, -1.1760],
         [-0.9534, -0.9705, -0.9192,  ..., -1.0219, -1.1075, -1.2274],
         ...,
         [-1.6727, -1.6727, -1.5185,  ...,  1.7352,  1.6495,  1.7865],
         [-1.6727, -1.6555, -1.6384,  ...,  1.6838,  1.5982,  1.7009],
         [-1.6384, -1.7240, -1.6727,  ...,  1.7180,  1.7352,  1.6838]],

        [[-0.2150, -0.1099, -0.1099,  ..., -0.5126, -0.4776, -0.6352],
         [-0.1975, -0.1625, -0.1625,  ..., -0.5126, -0.5476, -0.6001],
         [-0.0749, -0.1275, -0.1450,  ..., -0.4776, -0.6001, -0.7052],
         ...,
         [-1.1429, -1.1429, -0.9678,  ...,  2.0609,  1.9734,  2.1134],
         [-1.1954, -1.1253, -1.0903,  ...,  2.0084,  1.9734,  2.0784],
         [-1.1954, -1.1954, -1.1429,  ...,  2.0434,  2.0784,  2.0609]],

        [[-1.4210, -1.2990, -1.2467,  ..., -1.8044, -1.7173, -1.7870],
         [-1.3513, -1.3164, -1.2641,  ..., -1

In [14]:
session

In [15]:
def predict(image):
    result = session.run([output_name], {input_name: image})
    return result[0][0].tolist()

In [16]:
import numpy as np

image_np = image_final.numpy()  # Convert PyTorch tensor to NumPy array
image_with_batch = np.expand_dims(image_np, axis=0) # Add batch dimension
prediction = predict(image_with_batch)

In [17]:
prediction

[0.09156641364097595]